# Import Modules

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
from qcodes.math_utils import FieldVector
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes_contrib_drivers.drivers.Tektronix.Keithley_2700 import Keithley_2700
from qcodes.instrument_drivers.tektronix.Keithley_2600_channels import Keithley_2600
from qcodes_contrib_drivers.drivers.Oxford.IPS120 import OxfordInstruments_IPS120
from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.instrument_drivers.stanford_research.SR860 import SR860
from qcodes.dataset.plotting import plot_dataset
from qcodes.instrument_drivers.oxford.MercuryiPS_VISA import MercuryiPS
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

# Connect to Instruments and Initialize Variables

In [2]:
#SIM Controller
sim=SIM928('George','GPIB1::15::INSTR')
#Lockins(SR860 and SR830)
#diff1=SR830('Dilbert','GPIB1::6::INSTR' )
diff1=SR860('Laughlin', 'GPIB1::4::INSTR')

Connected to: Stanford_Research_Systems SIM900 (serial:s/n130213, firmware:ver3.6) in 1.26s
Connected to: Stanford_Research_Systems SR860 (serial:004479, firmware:V1.51) in 0.05s


# Global Parameters for Instruments and Experiment

In [3]:
dataBasePath = 'C:\\Users\\Admin\Desktop\\MattMann\\8-3-22.1MM11A3\\DifferentialResistance2\\DifferentialResistance_092923_nu=2.db'


In [70]:
csvPath = "C:\\Users\\Admin\\Desktop\\MattMann\\8-3-22.1MM11A3\\DifferentialResistance2\\DifferentialResistance_Contact4_100pA_nu=2_SweepDown.csv"
sample = "8-3-22.1MM11A3"
exp_name = "Differential_Conductance_100pA_Contact4_SweepDown"
meas_name = 'Differential_Conductance_100pA'
dc_amp = 0.1
amp = 0.000001
#Module number on SIM controller
sim_port = 1
#Units are in volts
V1_start = sim.get_voltage(sim_port)
V1_end = -0.5
#Ramp rate is volts/sec
rampRate = 0.002
#Changes direction of ramp if ramping down to a voltage instead of up
if V1_end < V1_start:
    rampRate = rampRate *(-1)

# Set Up Database for Data Collection

In [5]:
db_file_path = os.path.join(os.getcwd(), dataBasePath)
initialise_or_create_database_at(db_file_path)

# Set Up Live Plot

In [6]:
#Opens windows that will plot results as they are collected
#Make sure to turn on refresh rate once window opens or nothing will happen
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x000000001D5410D8>>

# Setup Station and Measurement Parameters

In [7]:
#Handles equipment
station = qc.Station()
station.add_component(sim)
station.add_component(diff1)

'Laughlin'

In [71]:
#Manages independent and dependent variables
diff_Conduct = load_or_create_experiment(
    experiment_name=exp_name,
    sample_name=sample
)
context_meas = Measurement(exp=diff_Conduct, station=station, name=meas_name)
context_meas.write_period = 0.1

context_meas.register_parameter(sim.volt_1)
context_meas.register_parameter(diff1.X, setpoints=(sim.volt_1,))

# Sweep DC Voltage and Measure Differential Conductance

In [ ]:
with context_meas.run() as datasaver:
    for v_set in np.arange(V1_start, V1_end+rampRate, rampRate):
        sim.set_voltage(sim_port, v_set)
        get_x1 = diff1.X.get()/amp
        datasaver.add_result((sim.volt_1, v_set/dc_amp),
                             (diff1.X, get_x1))        
        time.sleep(1)
    dataset = datasaver.dataset

Starting experimental run with id: 17. 


# Converts Dataset Results to CSV

In [66]:
#Converts to pandas dataframe
df = dataset.to_pandas_dataframe()
#Converts dataframe to csv
df.to_csv(csvPath)

# Plot Dataset Results

In [ ]:
plot_dataset(dataset)